# Unsloth: Reinforcement Learning (DPO/ORPO)

This notebook demonstrates how to perform **Direct Preference Optimization (DPO)** or **Odds Ratio Preference Optimization (ORPO)** using Unsloth.

These techniques align the model with human preferences using a dataset containing 'chosen' and 'rejected' responses.

In [ ]:
%%capture
!pip install unsloth
!pip install --no-deps "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
from unsloth import FastLanguageModel, PatchDPOTrainer
from unsloth import is_bfloat16_supported
import torch

# Patch DPO Trainer to make it work with Unsloth
PatchDPOTrainer()

max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/zephyr-sft-bnb-4bit", # Starting with an SFT model is best for DPO
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

### LoRA Configuration
We add LoRA adapters as usual.

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

### Data Preparation
We use `mlabonne/orpo-dpo-mix-40k`, which contains `chosen` and `rejected` columns.

In [ ]:
from datasets import load_dataset
dataset = load_dataset("mlabonne/orpo-dpo-mix-40k", split = "train")

# DPO requires specific column names
def format_dpo(examples):
    return {
        "prompt": examples["instruction"],
        "chosen": examples["chosen"],
        "rejected": examples["rejected"],
    }
dataset = dataset.map(format_dpo, batched = True)

In [ ]:
from trl import DPOTrainer
from transformers import TrainingArguments

dpo_trainer = DPOTrainer(
    model = model,
    ref_model = None, # Unsloth handles this efficiently without loading a second model
    tokenizer = tokenizer,
    beta = 0.1,
    train_dataset = dataset,
    max_length = max_seq_length,
    max_prompt_length = 512,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 5e-6, # Lower learning rate for DPO
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

In [ ]:
dpo_trainer.train()